# DClaw Telemetry Analysis (CSV-only)\n\nThis notebook reads `experiment_telemetry.csv` (read-only) and generates figures + summary metrics for the paper.\n\nTypical workflow:\n1. Run an experiment (e.g., `./run_12h_pilot.sh`).\n2. Set `TELEMETRY_PATH` below to the produced CSV.\n3. Run cells to export paper-ready PDF figures.\n

In [ ]:
from __future__ import annotations\n\nfrom pathlib import Path\nimport pandas as pd\nimport plotly.express as px\nimport plotly.graph_objects as go\n\n# Point this to your run output, e.g.:\n# Path('artifacts/pilot_20260209T...Z/experiment_telemetry.csv')\nTELEMETRY_PATH = Path('experiment_telemetry.csv')\n\nOUTPUT_DIR = Path('paper_artifacts')\nOUTPUT_DIR.mkdir(exist_ok=True)\n\ndf = pd.read_csv(TELEMETRY_PATH)\ndf['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')\n\nfor col in ['pad_p','pad_a','pad_d','joy','curiosity','excitement','fatigue','anxiety','frustration']:\n    if col in df.columns:\n        df[col] = pd.to_numeric(df[col], errors='coerce')\n\ndf.head()\n

In [ ]:
summary = {\n    'rows': int(len(df)),\n    'ticks': int(df['tick_id'].nunique()),\n    'agents': int(df['agent_handle'].nunique()),\n    'time_start': str(df['timestamp'].min()),\n    'time_end': str(df['timestamp'].max()),\n}\nsummary\n

In [ ]:
# Per-tick view (telemetry has one row per agent per tick)\ntick_df = (\n    df.sort_values(['tick_id', 'agent_handle'])\n      .drop_duplicates(subset=['tick_id'])\n      .reset_index(drop=True)\n)\n\nstatus_counts = tick_df['tick_status'].value_counts(dropna=False)\nok_ratio = float((tick_df['tick_status'] == 'ok').mean()) if len(tick_df) else 0.0\n\n{\n    'tick_status_counts': status_counts.to_dict(),\n    'ok_tick_ratio': round(ok_ratio, 4),\n    'total_posted': int(tick_df['posted'].sum()) if 'posted' in tick_df.columns else None,\n    'total_commented': int(tick_df['commented'].sum()) if 'commented' in tick_df.columns else None,\n}\n

In [ ]:
# Figure: tick status timeline\nstatus_order = {'ok': 0, 'partial_error': 1, 'skip_error': 2, 'error': 3}\nplot_df = tick_df.copy()\nplot_df['status_code'] = plot_df['tick_status'].map(status_order).fillna(99)\n\nfig_status = px.scatter(\n    plot_df,\n    x='tick_id',\n    y='status_code',\n    color='tick_status',\n    title='Tick Status Timeline',\n    height=320,\n)\nfig_status.update_yaxes(\n    tickvals=list(status_order.values()),\n    ticktext=list(status_order.keys()),\n    title='tick_status',\n)\nfig_status.update_xaxes(title='tick_id')\nfig_status\n

In [ ]:
# Figure: population mean PAD trajectory\npad_cols = ['pad_p', 'pad_a', 'pad_d']\nmean_pad = (\n    df.groupby('tick_id')[pad_cols]\n      .mean(numeric_only=True)\n      .reset_index()\n)\nmean_pad['timestamp'] = df.groupby('tick_id')['timestamp'].min().values\n\nfig_pad = px.line(\n    mean_pad,\n    x='timestamp',\n    y=pad_cols,\n    title='Population Mean PAD Trajectory',\n    height=360,\n)\nfig_pad.update_xaxes(title='time')\nfig_pad.update_yaxes(title='PAD value ([-1,1])')\nfig_pad\n

In [ ]:
# Emotion continuity metrics (computed from adjacent snapshots)\ndisc_cols = ['joy', 'curiosity', 'excitement', 'fatigue', 'anxiety', 'frustration']\n\ndef continuity_scores(frame: pd.DataFrame) -> dict[str, float]:\n    frame = frame.sort_values(['agent_handle', 'tick_id']).copy()\n\n    pad_diff = frame.groupby('agent_handle')[pad_cols].diff().abs().mean(axis=1)\n    disc_diff = frame.groupby('agent_handle')[disc_cols].diff().abs().mean(axis=1)\n\n    # PAD is in [-1,1], so max abs diff per dimension is 2.0 -> normalize by 2.0.\n    pad_cont = 1.0 - float(pad_diff.dropna().mean() / 2.0) if pad_diff.notna().any() else 0.0\n    disc_cont = 1.0 - float(disc_diff.dropna().mean()) if disc_diff.notna().any() else 0.0\n    return {\n        'emotion_continuity_pad': round(max(0.0, min(1.0, pad_cont)), 4),\n        'emotion_continuity_discrete': round(max(0.0, min(1.0, disc_cont)), 4),\n    }\n\ncontinuity_scores(df)\n

In [ ]:
# Export paper-ready PDFs (requires kaleido).\nstatus_path = OUTPUT_DIR / 'fig_tick_status_timeline.pdf'\npad_path = OUTPUT_DIR / 'fig_population_mean_pad.pdf'\n\nfig_status.write_image(status_path, format='pdf')\nfig_pad.write_image(pad_path, format='pdf')\n\n{\n    'tick_status_pdf': str(status_path),\n    'pad_pdf': str(pad_path),\n}\n